# ConvLSTM functional encoder/decoder

In [9]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [10]:
print("tensorflow version:",tf.__version__)

tensorflow version: 2.0.0


## Input dimension variables

In [11]:
frames = 1
channels = 1
pixels_x = 21
pixels_y = 21

## Define the model

NOTE: Old versions of Tensorflow (<2.2) may not have support for initial_state on ConvLSTM2D cells. 
Please follow the instructions here: https://stackoverflow.com/questions/50253138/convlstm2d-initial-state-assertion-error to fix it.

In [12]:
model_name = 'encoder_convlstm_'+str(frames)+'f_'+str(channels)+'c_'+str(pixels_x)+'x_'+str(pixels_y)+'y'
print(model_name)

encoder_convlstm_1f_1c_21x_21y


In [15]:
##### ENCODER #####

# first time-step
i = 0
# get input_images and output_images as one tensor
input1 = layers.Input(name="encoder_input{}".format(i+1),
                      shape = (2*frames, channels, pixels_x, pixels_y))
input_images, output_images = tf.unstack(input1, num=None, axis=1, name='unstack')
input_images = tf.expand_dims(input_images, 0)
output_images = tf.expand_dims(output_images,0)
noise = layers.GaussianNoise(0.1)(input_images)
encoder_cell_1 = layers.ConvLSTM2D(name="encoder{}".format(i+1),
                                   filters = channels,
                                   kernel_size=(5,5),
                                   padding='same',
                                   data_format='channels_first',
                                   return_sequences=True,
                                   return_state=True)
_, state_h, state_c = encoder_cell_1(noise)

encoder_states = [state_h, state_c]

##### DECODER #####

# input2 = layers.Input(name="decoder_input{}".format(i+1),
#                       shape = (frames, channels, pixels_x, pixels_y))
decoder_cell_1 = layers.ConvLSTM2D(name="decoder{}".format(i+1),
                                   filters = channels,
                                   kernel_size=(5,5),
                                   padding='same',
                                   data_format='channels_first',
                                   return_sequences=True,
                                   return_state=True)
decoder_output, _, _ = decoder_cell_1(output_images, initial_state = encoder_states)

##### COLLECT AND COMPILE #####
encoder_stack = keras.Model(name='encodeco_1',
                            inputs = [input1, input2], 
                            outputs = decoder_output
                         )

encoder_stack.compile(loss='mean_squared_error',
                  optimizer='adadelta',
                  metrics=['accuracy', 'mean_absolute_error'])
encoder_stack.summary()

Model: "encodeco_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input1 (InputLayer)     [(None, 2, 1, 21, 21 0                                            
__________________________________________________________________________________________________
tf_op_layer_unstack_2 (TensorFl [(None, 1, 21, 21),  0           encoder_input1[0][0]             
__________________________________________________________________________________________________
tf_op_layer_ExpandDims (TensorF [(1, None, 1, 21, 21 0           tf_op_layer_unstack_2[0][0]      
__________________________________________________________________________________________________
gaussian_noise_3 (GaussianNoise (1, None, 1, 21, 21) 0           tf_op_layer_ExpandDims[0][0]     
_________________________________________________________________________________________

## Pickle the model

In [17]:
tf.keras.models.save_model(
    model = encoder_stack,
    filepath = '../models/'+model_name+'.h5',
    overwrite=True,
    include_optimizer=True,
    save_format='tf',
    signatures=None
)

In [14]:
# returns train, inference_encoder and inference_decoder models
def define_models(frames, channels, pixels_x, pixels_y):
    i=0
    n_output = channels * pixels_x * pixels_y
    # define training encoder
    encoder_inputs = layers.Input(name="encoder_input{}".format(i+1),
                      shape = (frames, channels, pixels_x, pixels_y))
    encoder = layers.ConvLSTM2D(name="encoder{}".format(i+1),
                                   filters = channels,
                                   kernel_size=(5,5),
                                   padding='same',
                                   strides=(1,1),
                                   data_format='channels_first',
                                   return_sequences=True,
                                   return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    # define training decoder
    decoder_inputs = layers.Input(name="decoder_input{}".format(i+1),
                      shape = (frames, channels, pixels_x, pixels_y))
    decoder_lstm = layers.ConvLSTM2D(name="decoder{}".format(i+1),
                                   filters = channels,
                                   kernel_size=(5,5),
                                   padding='same',
                                   strides=(1,1),
                                   data_format='channels_first',
                                   return_sequences=True,
                                   return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    
    decoder_dense = layers.Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    # define inference encoder
    encoder_model = keras.Model(encoder_inputs, encoder_states)
    # define inference decoder
    decoder_state_input_h = layers.Input(name="hidden_decoder_input{}".format(i+1),
                      shape = (frames, channels, pixels_x, pixels_y))
    decoder_state_input_c = layers.Input(name="cell_decoder_input{}".format(i+1),
                      shape = (frames, channels, pixels_x, pixels_y))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    # return all models
    return model, encoder_model, decoder_model
    

In [15]:
model, encoder_model, decoder_model = define_models(frames, channels, pixels_x, pixels_y)

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory